* __数据集__:GroupLens提供的MovieLens数据集

* __实验设计__:

In [51]:
import pandas as pd
import random
import numpy as np

In [44]:
movies_columns = ['id', 'name', 'performer']
movies = pd.read_table('movies.dat', encoding='iso-8859-1', sep='::', names=movies_columns)
movies.head()

/Users/pengyi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,id,name,performer
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [40]:
ratings_columns = ['user', 'movie', 'start', 'timestamp']
ratings = pd.read_table('ratings.dat', encoding='iso-8859-1', sep='::', names=ratings_columns)
ratings.head()

/Users/pengyi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,user,movie,start,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [45]:
'''
性别由男性的“M”和女性的“F”来表示。

年龄从以下范围选择：
* 1：“18岁以下”
＊18：“18-24”
＊25：“25-34”
＊35：“35-44”
＊45：“45-49”
* 50：“50-55”
* 56：“56 +”

职业是从以下选择中选择的：
* 0：“其他”或未指定
* 1：“学术/教育家”
* 2：“艺术家”
＊3：“文书/行政”
* 4：“大学/研究生”
＊5：“客户服务”
* 6：“医生/保健”
* 7：“执行/管理”
* 8：“农民”
* 9：“家庭主妇”
* 10：“K-12学生”
* 11：“律师”
* 12：“程序员”
* 13：“退休”
* 14：“销售/营销”
* 15：“科学家”
* 16：“个体经营”
* 17：“技术员/工程师”
* 18：“商人/工匠”
＊19：“失业”
* 20：“作家”
'''
user_columns = ['id', 'sex', 'age', 'occupation', 'zip_code']
users = pd.read_table('users.dat', encoding='iso-8859-1', sep='::', names=user_columns)
users.head()

/Users/pengyi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,id,sex,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [76]:
# 切分用户行为数据数据为训练集和测试集
def SplitData(X,test_size=0.3):
    X_num=X.shape[0]
    train_index=list(range(X_num))
    test_index=[]
    test_num=int(X_num*test_size)
    for i in range(test_num):
        randomIndex=int(np.random.uniform(0,len(train_index)))
        test_index.append(train_index[randomIndex])
        del train_index[randomIndex]
    #train,test的index是抽取的数据集X的序号
    train=X.ix[train_index] 
    test=X.ix[test_index]
    return train,test
train, test = SplitData(ratings)
train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [88]:
# 计算召回率
def Recall(train, test, N):
    hit = 0
    all = 0
    for user in train.user:
        tu = test(user)
        rank = GetRecommendation(user, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += N
    return hit/(all * 1.0)

In [90]:
# 计算准确率
def Precision(train, test, N):
    hit = 0
    all = 0
    for user in train.user:
        tu = test(user)
        rank = GetRecommendation(user, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += 1
    return hit/(all * 1.0)

## 基于用户的协同过滤算法

In [94]:
# 计算余弦相似度
def UserSimilarity(trian):
    w - dict()
    for u in train.keys():
        for v in train.keys:
            if u == v:
                continue
            W[u][v] = len(train[u] & train[v])
            # c/=a 等效于 c = c/a
            W[u][v] /= math.sqrt(len(trian[u]) * len(trian[v] * 1))
    return W

# 利用物品到用户的倒排表,计算余弦相似度
def UserSimilarity(trian):
    item_users = dict()
    for u, items in train.items():
        for i in items.keys():
            if i not in item_users:
                item_users[i] = set()
            item_users[i].add(u)
    c = dict()
    n = dict()
    for i, users in item_users.items():
        for u in users:
            n[u] += 1
            for v in users:
                if u == v:
                    continue
                c[u][v] += 1
    w = dict()
    for u, related_users in c.items():
        for v, cuv in related_users.items():
            w[u][v] =cuv / math.sqrt(n[u] * n[v])
    return w

In [106]:
# UserCF推荐算法计算用户u对物品i的感兴趣程度
def Recommend(user, train, W, K):
    rank = dict()
    interacted_items = train[user]
    for v, wuv in sorted(W[u].items, key=itemgetter(1), reverse=True)[0:K]:
        for i, rvi in train[v].items:
            if i in iteracted_itmes:
                continue
            rank[i] += wuv * rvi
    return rank

In [103]:
# User-IIF算法
def UserSimilarity(train):
    # 建立物品用户倒排表
    for u, items in train.items():
        for i in tiems.keys():
            if i not in items.users:
                item_users[i] = set()
    c - dict()
    n = dict()
    # 计算各个物品的用户相似评级
    for i, user in item_users.items():
        for u in users:
            N[u] += 1
            for v in users:
                if u == v:
                    continue
                c[u][v] += 1 / math.log(1 + len(users))
    w = dict()
    # 计算用户相似度矩阵
    for u, related_users in c.items():
        for v, cuv in related_users.items():
            w[u][v] = cuv / math.sqrt(n[u] * n[v])
    return w

## 基于物品的协同过滤算法

In [105]:
# 利用ItemCF算法计算物品相似度
def ItemSimilarity(trian):
    # 建立物品倒排表
    c = dict()
    n = dict()
    for u, items in train.items():
        for i in users:
            n[i] += 1
            for i in users:
                n[i] += 1
                for j in users:
                    if i==j:
                        continue
                    c[i][j] += 1
    # 计算相似度矩阵
    w = dict()
    for i, related_tiems in c.items():
        for j, cij in related_times.items():
            w[u][v] = cij / math.sqrt(n[i] * n[j])
    return w

In [108]:
# 得到物品相似度后,ItemCF通过以下方法计算一个用户对物品j的兴趣
def Recommentdation(train, user_id, w, k):
    rank = dict()
    ru = train(user_id)
    for i, pi in ru.items():
        for j, wj in sorted(w[i].items, key=itemgetter(1), reverse=True)[0:K]:
            if j in ru:
                continue
            rank[j] += pi * wj
    return rank

## 隐语义模型

In [ ]:
# 实现负样本采样过程
'''
items为用户有过行为的物品
items_pool为用户没有行为的候选物品
'''
def RandomSelectNegativeSample(self, items):
    ret = dict()
    for i in items.keys():
        ret[i] = 1
    n = 0
    for in in range(0, len(item) * 3):
        item = items_pool[random.randint(0, len(items_pool) -1)]
        if item in ret:
            comtinue
        ret[item] = 0
        n += 1
        if n > len(items):
            break
    return ret

In [ ]:
# 随机梯度下降
def LatentFactorModel(user_items, F, N, alpha, lambda):
    [P, Q] = InitModel(user_items, F)
    for step in range(0, N):
        for user, items in user_items.items():
            samples = RandSelectNegativeSamples(items)
            for item, rui in samples.items():
                eui = rui - Predict(user, item)
                for f in range(0, F):
                    P[user][f] += alpha * (eui * Q[item][f] - lambda * P[user][f])
                    Q[item][f] += alpha * (eui * P[user][f] - lambda * Q[item][f])
        alpha *= 0.9

def Recommend(user, P, Q):
    rank = dict()
    for f, puf in p[user].items():
        for i, qfi in Q[f].items():
            if i not in rank:
                rank[i] += puf * qfi
    return rank

## 基于图的推荐算法

### 度量图中两个顶点的相关性方法
* 两个顶点的路径数
* 两个顶点的路径长度
* 两个顶点之间的路径经过的顶点

In [ ]:
# 基于随机游走的PersonalRank算法
def PersonalRank(G, alpha, root):
    rank = dict()
    rank = (x:0 for x in G.keys())
    rank[root] = 1
    for k in range(20):
        tmp = (x:0 for x in G.keys())
        for i, ri in G.items():
            for j, wij in ri.items():
                if j not in tmp:
                    tmp[j] = 0
                tmp[j] += 0.6 rank[i] / (1.0 * len(ri))
                if j == root:
                    tmp[j] += 1 - alpha
        rank = tmp
    rank = rank